# Assignment 3
## Econ 8310 - Business Forecasting

For homework assignment 3, you will work with [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist), a more fancier data set.

- You must create a custom data loader as described in the first week of neural network lectures [2 points]
    - You will NOT receive credit for this if you use the pytorch prebuilt loader for Fashion MNIST!
- You must create a working and trained neural network using only pytorch [2 points]
- You must store your weights and create an import script so that I can evaluate your model without training it [2 points]

Highest accuracy score gets some extra credit!

Submit your forked repository URL on Canvas! :) I'll be manually grading this assignment.

Some checks you can make on your own:
- Did you manually process the data or use a prebuilt loader (see above)?
- Does your script train a neural network on the assigned data?
- Did your script save your model?
- Do you have separate code to import your model for use after training?

In [2]:
import os
import gzip
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import urllib.request

# Fetch Fashion MNIST files manually
def fetch_fashion_data():
    url = "http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/"
    file_dict = {
        "train_img": "train-images-idx3-ubyte.gz",
        "train_lbl": "train-labels-idx1-ubyte.gz",
        "test_img": "t10k-images-idx3-ubyte.gz",
        "test_lbl": "t10k-labels-idx1-ubyte.gz"
    }
    os.makedirs("data", exist_ok=True)
    for key, filename in file_dict.items():
        path = os.path.join("data", filename)
        if not os.path.isfile(path):
            urllib.request.urlretrieve(url + filename, path)

# Load image data
def parse_images(filepath):
    with gzip.open(filepath, 'rb') as f:
        f.read(16)
        images = np.frombuffer(f.read(), dtype=np.uint8).reshape(-1, 28*28)
    return images.astype(np.float32) / 255.0

# Load label data
def parse_labels(filepath):
    with gzip.open(filepath, 'rb') as f:
        f.read(8)
        return np.frombuffer(f.read(), dtype=np.uint8)

# Custom Dataset class
class FMNISTDataset(Dataset):
    def __init__(self, images, labels):
        self.X = torch.tensor(images)
        self.Y = torch.tensor(labels)

    def __len__(self):
        return len(self.Y)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

# MLP Neural Network
class FashionClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(784, 128),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        return self.model(x)

def train_and_save():
    fetch_fashion_data()
    x_train = parse_images("data/train-images-idx3-ubyte.gz")
    y_train = parse_labels("data/train-labels-idx1-ubyte.gz")
    x_test = parse_images("data/t10k-images-idx3-ubyte.gz")
    y_test = parse_labels("data/t10k-labels-idx1-ubyte.gz")

    train_data = FMNISTDataset(x_train, y_train)
    test_data = FMNISTDataset(x_test, y_test)

    train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=64)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = FashionClassifier().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for ep in range(1, 11):
        model.train()
        cumulative_loss = 0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            preds = model(xb)
            loss = criterion(preds, yb)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            cumulative_loss += loss.item()
        print(f"Epoch {ep} | Training Loss: {cumulative_loss / len(train_loader):.4f}")

    torch.save(model.state_dict(), "saved_fmnist_model.pth")
    print("Model saved successfully as 'saved_fmnist_model.pth'.")

    # Accuracy on test data
    model.eval()
    total, correct = 0, 0
    with torch.no_grad():
        for xb, yb in test_loader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            _, predicted = torch.max(out, 1)
            correct += (predicted == yb).sum().item()
            total += yb.size(0)

    print(f"Final Test Accuracy: {100 * correct / total:.2f}%")

if __name__ == "__main__":
    train_and_save()

Epoch 1 | Training Loss: 0.5928
Epoch 2 | Training Loss: 0.4182
Epoch 3 | Training Loss: 0.3819
Epoch 4 | Training Loss: 0.3590
Epoch 5 | Training Loss: 0.3431
Epoch 6 | Training Loss: 0.3337
Epoch 7 | Training Loss: 0.3232
Epoch 8 | Training Loss: 0.3110
Epoch 9 | Training Loss: 0.3064
Epoch 10 | Training Loss: 0.2999
Model saved successfully as 'saved_fmnist_model.pth'.
Final Test Accuracy: 88.00%


In [3]:
import os
import gzip
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

# Load image data
def parse_images(filepath):
    with gzip.open(filepath, 'rb') as f:
        f.read(16)
        images = np.frombuffer(f.read(), dtype=np.uint8).reshape(-1, 28*28)
    return images.astype(np.float32) / 255.0

# Load label data
def parse_labels(filepath):
    with gzip.open(filepath, 'rb') as f:
        f.read(8)
        return np.frombuffer(f.read(), dtype=np.uint8)

# Custom Dataset class
class FMNISTDataset(Dataset):
    def __init__(self, images, labels):
        self.X = torch.tensor(images)
        self.Y = torch.tensor(labels)

    def __len__(self):
        return len(self.Y)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

# Same model structure as training
class FashionClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(784, 128),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        return self.model(x)

def load_and_evaluate():
    x_test = parse_images("data/t10k-images-idx3-ubyte.gz")
    y_test = parse_labels("data/t10k-labels-idx1-ubyte.gz")
    test_data = FMNISTDataset(x_test, y_test)
    test_loader = DataLoader(test_data, batch_size=64)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = FashionClassifier().to(device)
    model.load_state_dict(torch.load("saved_fmnist_model.pth", map_location=device))
    model.eval()

    total, correct = 0, 0
    with torch.no_grad():
        for xb, yb in test_loader:
            xb, yb = xb.to(device), yb.to(device)
            preds = model(xb)
            _, predicted = torch.max(preds, 1)
            correct += (predicted == yb).sum().item()
            total += yb.size(0)

    print(f"Test Accuracy (loaded model): {100 * correct / total:.2f}%")

if __name__ == "__main__":
    load_and_evaluate()

Test Accuracy (loaded model): 88.00%
